In [1]:
from utils import *

In [2]:

# Ask user for league to scrape

# Main Script
if __name__ == '__main__':
    # Get team name from user
    league = input("Enter the league: ")
    gender = input("Enter the competition gender (M/F): ")
    folder_path = os.path.join(os.getcwd(), "Competitions")
    os.makedirs(folder_path, exist_ok=True)  # Create the folder if it doesn't exist
    # File-based caching
    LEAGUES_CACHE = folder_path + '\league_links.json'
    closest_league, league_info = get_closest_league(league,LEAGUES_CACHE,gender)

    if closest_league:
        print(f"Closest match: {closest_league}, URL: {league_info['url']}")
    else:
        print("No close match found.")

    # Select season 
    season = input("Select the season")
    # get selected competition folder (or create it)
    if gender=='M':
        full_gender = 'Men'
    else:
        full_gender = 'Women'
    folder_path = os.path.join(os.getcwd(), closest_league + '-' + full_gender)
    os.makedirs(folder_path, exist_ok=True)  
    SEASONS_CACHE = os.path.join(folder_path, f'{closest_league}_seasons.json')
    season_url = get_season_url(season,SEASONS_CACHE,league_info['url'])

    if season_url:
        print(f"Closest match: {season}, URL: {season_url[1]}")
    else:
        print("Season URL not found")
    
    # Get tables from the competition page
    competition_url = season_url[1]
    folder_path = folder_path + f"\{season}"
    os.makedirs(folder_path, exist_ok=True)
    output_file_for = folder_path + "\Season-Stats.xlsx"
    table_card_position = "left"
    scrape_page_tables(competition_url, output_file_for,table_card_position)
    print(f"{closest_league} data for team on the {season} saved in {output_file_for}.")

    respect_fbref_scrape_policy() # 6 seconds timeout between requests

    output_file_vs = folder_path + "\Season-Stats-against.xlsx"
    table_card_position = "right"
    scrape_page_tables(competition_url, output_file_vs,table_card_position)
    print(f"{closest_league} data vs team on the {season} saved in {output_file_vs}.")

    # Move to the Fixture page
    table_card_position = "left"
    output_file = folder_path + "\Fixtures.xlsx"
    fixtures_url = get_scores_and_fixtures_url(competition_url)
    print(f"Fixture URL: {fixtures_url}")
    scrape_page_tables(fixtures_url,output_file,table_card_position)

    respect_fbref_scrape_policy() # 6 seconds timeout between requests

    # Extract league teams urls
    team_urls = []
    team_urls = extract_team_urls(competition_url)
    print(f"Team URLS: {team_urls}")
    match_report_urls = []
    for entry in team_urls:
        
        match_report_urls = extract_match_report_urls(entry['team'],entry['url'],closest_league)
        print(f"Match Report URLS: {match_report_urls}")

        # Eliminate empty rows and update fixture file with match report URLs
        df = pd.read_excel(output_file, engine='openpyxl')
        df_cleaned = df.dropna(how='all')
        team = match_report_urls[0]['team'] # get the team  from the first entry
        urls = [url_entry['url'] for url_entry in match_report_urls]
        update_fixtures_with_match_report_urls(df_cleaned,team,urls,output_file)
        
        # Create a folder for each team inside the root folder path
        team_folder_path = os.path.join(folder_path, f"{entry['team']} - Match Reports")
        os.makedirs(team_folder_path, exist_ok=True)

        #folder_path = folder_path + f"\{entry['team']} - Match Reports"
        #os.makedirs(folder_path, exist_ok=True)

        match_number = 1
        for report_url in match_report_urls:
                       print(f"Report_url: {report_url}")
                       report_file = os.path.join(team_folder_path, f"Match {match_number} {report_url['team']} - {report_url['opponent']}.xlsx")
                       #report_file = folder_path + f"\Match {match_number} {report_url['team']} - {report_url['opponent']}.xlsx" 
                       respect_fbref_scrape_policy() # commented out because I am calling the function in the scrape_and_save_reports workflow
                       scrape_and_save_reports(report_url['url'],report_file,match_number,report_url['team'],report_url['opponent'])
                       match_number += 1
                       
    print(f"Extracted URLs: {match_report_urls}") #debug




Closest match: FA Women's Super League, URL: https://fbref.com/en/comps/189/history/Womens-Super-League-Seasons
Closest match: 2023-2024, URL: https://fbref.com/en/comps/189/2023-2024/2023-2024-Womens-Super-League-Stats
FA Women's Super League data for team on the 2023-2024 saved in c:\Users\Francesco\Fbref-scraping\FA Women's Super League-Women\2023-2024\Season-Stats.xlsx.
Respecting FBref scrape policy... Sleeping for 6 seconds.
FA Women's Super League data vs team on the 2023-2024 saved in c:\Users\Francesco\Fbref-scraping\FA Women's Super League-Women\2023-2024\Season-Stats-against.xlsx.
Fixture URL: https://fbref.com/en/comps/189/2023-2024/schedule/2023-2024-Womens-Super-League-Scores-and-Fixtures
Respecting FBref scrape policy... Sleeping for 6 seconds.
Table: <table class="stats_table sortable min_width" data-cols-to-freeze=",1" id="stats_squads_standard_for"> <caption>Squad Standard Stats <span style="color: #666; font-size:smaller">2023-2024 Women's Super League</span> Table</